# Experiment description
## Hypothesis: 
General predictability values related to rnmc database are similar to the ones reported from nuse analysis (experiment 08)
## Method: 
- Select rnmc registers 2017-2018
- Remove outliers > q=0.99, assign new max values to outliers
- Assign 0 for empty dates
- Compare predictability values with the ones obtained on experiment 8.

## Built-in methods

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import scipy
import math
from math import pi
import geopandas as gpd
#from matplotlib.collections import PatchCollection
%matplotlib inline

In [ ]:
workingPath= '/Users/anamaria/Desktop/dev/security_project/'

In [ ]:
def set_initial_dataset_day(df_by_date,name_day):
    df_by_date = df_by_date.reset_index()
    df_by_date['day_of_week'] = df_by_date['date'].dt.day_name()
    monday_idx = df_by_date.index[df_by_date['day_of_week'] == name_day].tolist()[0]
    df_by_date = df_by_date[monday_idx:].set_index('date').drop(['day_of_week'],axis=1)
    return df_by_date

In [ ]:
# Methods for time windows
def im2patches(im,n):
    patches = [];
    for i in range(len(im)-n):
        patch = im[i:(i+n-1)]        
        patch = patch - np.nanmean(patch);
        if(np.linalg.norm(patch)>0):
            patch = patch/np.linalg.norm(patch);
        if i==0:
            patches = patch;
        else:
            patches = np.vstack((patches,patch))
    return patches;

def writeEmbeding(timeSeries,lenWindow,samplePath, scenarioName):
    slicingWindows = im2patches(timeSeries,lenWindow);
    experimentPath = 'periodicity_experiments/predictability/slicing/'
    prevStation = str(samplePath);
    with open(workingPath+experimentPath+'slicingWindows'+"_"+str(prevStation)+"_"+str(scenarioName)+"_"+str(lenWindow)+'_.pickle', 'wb') as f:
        lv = slicingWindows.tolist();                        
        pickle.dump(lv, f, protocol=2)

    experimentPath = 'periodicity_experiments/predictability/timeSeries/'    
    with open(workingPath+experimentPath+'timeSeries'+"_"+str(prevStation)+"_"+str(scenarioName)+"_"+str(lenWindow)+'_.pickle', 'wb') as f:
        lv = timeSeries.tolist();                        
        pickle.dump(lv, f, protocol=2)

In [ ]:
#Methods for predictability
def getBarcode(samplePath,lenWindow,scenarioName):
    experimentPath = 'periodicity_experiments/predictability/'
    barcode = [];

    with open(workingPath+experimentPath+'timeSeries/'+'timeSeries_'+samplePath+"_"+str(scenarioName)+'_'+str(lenWindow)+'_'+'.pickle', 'rb') as f:
            timeSeries = pickle.load(f);            
    return (barcode,timeSeries);

def computeBarcodeEntropy(barsLenB0):
    barlen = np.array(barsLenB0);
    barlen = barlen/barlen.sum();
    hbc = 0;
    for i in range(barlen.shape[0]):
        if barlen[i]!=0:
            hbc = hbc-(barlen[i])*np.log(barlen[i]);
    return hbc;


def computeGeneralPredictability(timeSeries,binsData,lenWindow):
    # Colwell, R. K. (1974). Predictability, constancy, and contingency of periodic phenomena. Ecology, 55(5), 1148-1153.
    # Normalize the caudal values
    nLevels = binsData.shape[0]-1;
    matStations = np.array(timeSeries).reshape((np.array(timeSeries).shape[0]//lenWindow,lenWindow))    

    grandMean = np.mean(np.mean(matStations));
    #matStations = matStations / grandMean;
    N = np.zeros((nLevels,lenWindow));
    for i in range(1,matStations.shape[1]): 
        # Computes histograms per columns
        hist, bin_edges = np.histogram(matStations[:,i],bins = binsData);
        N[:,i] = hist;
    X = np.sum(N, axis=0);
    Y = np.sum(N, axis=1);
    Z = np.sum(Y);
    hx = 0;
    hy = 0;
    hxy = 0;
    for j in range(X.shape[0]):
        if X[j]!=0:
            hx = hx-(X[j]/Z)*np.log(X[j]/Z);
            
    for i in range(Y.shape[0]):
        if Y[i]!=0:
            hy = hy-(Y[i]/Z)*np.log(Y[i]/Z);
            
    for i in range(Y.shape[0]):
        for j in range(X.shape[0]):
            if N[i,j]!=0:
                hxy = hxy-((N[i,j]/Z)*np.log(N[i,j]/Z));    
    
    # predictability
    p = 1 - (hxy - hx)/np.log(N.shape[0]);
    # constancy
    c = 1 - hy/np.log(N.shape[0]);
    # Returns constancy and contingency
    return (c,p-c,p);



In [ ]:
def preprocess_df(df,min_date_period,max_date_period):
    df=df.drop(columns=['NOMBRE_LOCALIDAD'])
    #Remove outliers
    q_hi = df["total_eventos"].quantile(0.99)
    #df = df[(df["total_eventos"] < q_hi)]
    
    ### Comment this code section if previous line is uncommented
    df_without_outliers = df[(df["total_eventos"] < q_hi)]
    max_value = df_without_outliers['total_eventos'].max()
    df.loc[df["total_eventos"]>= q_hi,'total_eventos'] = max_value
    ###

    #Make sure dataset include consecutive dates in period
    idx = pd.date_range(min_date_period, max_date_period)
    #df = df.reindex(idx, fill_value=int(df["total_eventos"].mean()))
    df = df.reindex(idx, fill_value=int(0))
    df = df.reset_index().rename(columns={'index': 'date'}).set_index('date')
    
    #Make sure dataset starts on Monday for the experiment
    df = set_initial_dataset_day(df,'Monday')
    
    return df

In [ ]:
def saveTimeSeries(df,min_date_period,max_date_period,localidad, lenWindow, expName):       
    df_values = pd.Series(df['total_eventos']).values
    lT=get_LT(df, lenWindow)
    df_values = df_values[0:lT]
    writeEmbeding(df_values,lenWindow,expName,localidad)

In [ ]:
def get_LT(df_by_period,lenWindow):
    min_date = df_by_period.reset_index().date.min()
    max_date = df_by_period.reset_index().date.max()
    samples_num = (max_date.date()-min_date.date()).days
    lT = samples_num//lenWindow * lenWindow
    return lT

In [ ]:
def predictability_experiment_localidades(df_by_date,min_date_period,max_date_period,lenWindow,localidadesList,Levels,expName,periodName):
    #workingPath = '/Users/anamaria/Desktop/dev/security_project/periodicity_experiments/predictability/';

    flagF = True;
    for localidad in localidadesList:
        #write embeding
        df_by_localidad = df_by_date[df_by_date['NOMBRE_LOCALIDAD'] == localidad]        
        df_by_localidad = preprocess_df(df_by_localidad,min_date_period,max_date_period)
        #print(df_by_localidad["total_eventos"])
        saveTimeSeries(df_by_localidad,min_date_period,max_date_period,localidad, lenWindow, expName)
        
        for nLevels in Levels:
            (barcode,timeSeries) = getBarcode(expName,lenWindow,localidad);
            binsLevels = np.linspace(np.min(timeSeries),np.max(timeSeries),nLevels);
            c,m,p = computeGeneralPredictability(timeSeries,binsLevels,lenWindow)

            if flagF==True:
                flagF = False
                predValues = np.array([expName,periodName,localidad,lenWindow,nLevels,p,m,c]);
            else:
                predValues = np.vstack((predValues, [expName,periodName,localidad,lenWindow,nLevels,p,m,c]))

    return predValues

In [ ]:
def table_predictability_report(df_agressiveBehavior,lenWindow,localidadesList,levelCategories,name_experiment):
    join=df_agressiveBehavior.pivot('localidad','crime_level','predictability')
    var1_order = []
    var2_order = levelCategories
    if len(var2_order) > 0:
        join = join.reindex(var2_order, axis=1)
    if len(var1_order) > 0:
        join = join.reindex(var1_order)
    
    fig, ax = plt.subplots(1,1,sharex=True, sharey=True)
    fig.set_size_inches(7, 6)
    g=sns.heatmap(join.astype('float'),annot=True,fmt=".3",linewidths=0,cmap="Blues",cbar=False)
    g.set_yticklabels(g.get_yticklabels(), rotation = 0)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    file_path = 'periodicity_experiments/predictability/figures/'
    #plt.savefig(workingPath+file_path+'table_aggressiveBehavior_localidades'+str(name_experiment)+'_predictability_time_'+str(lenWindow)+'_levels_'+str(levelCategories),dpi=300,bbox_inches = "tight")
    plt.show()

In [ ]:
def table_constancy_report(df_agressiveBehavior,lenWindow,localidadesList,levelCategories,name_experiment):
    join=df_agressiveBehavior.pivot('localidad','crime_level','constancy')
    var1_order = []
    var2_order = levelCategories
    if len(var2_order) > 0:
        join = join.reindex(var2_order, axis=1)
    if len(var1_order) > 0:
        join = join.reindex(var1_order)
    
    fig, ax = plt.subplots(1,1,sharex=True, sharey=True)
    fig.set_size_inches(7, 6)
    g=sns.heatmap(join.astype('float'),annot=True,fmt=".3",linewidths=0,cmap="Blues",cbar=False)
    g.set_yticklabels(g.get_yticklabels(), rotation = 0)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    file_path = 'periodicity_experiments/predictability/figures/'
    plt.savefig(workingPath+file_path+'table_aggressiveBehavior_localidades'+str(name_experiment)+'_constancy_time_'+str(lenWindow)+'_levels_'+str(levelCategories),dpi=300,bbox_inches = "tight")
    plt.show()

In [ ]:
def table_contingency_report(df_agressiveBehavior,lenWindow,localidadesList,levelCategories,name_experiment):
    join=df_agressiveBehavior.pivot('localidad','crime_level','contingency')
    var1_order = []
    var2_order = levelCategories
    if len(var2_order) > 0:
        join = join.reindex(var2_order, axis=1)
    if len(var1_order) > 0:
        join = join.reindex(var1_order)
    
    fig, ax = plt.subplots(1,1,sharex=True, sharey=True)
    fig.set_size_inches(7, 6)
    g=sns.heatmap(join.astype('float'),annot=True,fmt=".3",linewidths=0,cmap="Blues",cbar=False)
    g.set_yticklabels(g.get_yticklabels(), rotation = 0)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    file_path = 'periodicity_experiments/predictability/figures/'
    plt.savefig(workingPath+file_path+'table_aggressiveBehavior_localidades'+str(name_experiment)+'_contingency_time_'+str(lenWindow)+'_levels_'+str(levelCategories),dpi=300,bbox_inches = "tight")
    plt.show()

In [ ]:
def all_predictability_measures(df,lenWindow,crime_level,name_experiment):
    join=df.pivot('localidad','variable','value')
    var1_order = []
    var2_order = []
    if len(var2_order) > 0:
        join = join.reindex(var2_order, axis=1)
    if len(var1_order) > 0:
        join = join.reindex(var1_order)

    fig, ax = plt.subplots(1,1,sharex=True, sharey=True)
    fig.set_size_inches(7, 6)
    g=sns.heatmap(join.astype('float'),annot=True,fmt=".3",linewidths=0,cmap="Blues",cbar=False)
    g.set_yticklabels(g.get_yticklabels(), rotation = 0)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    file_path = 'periodicity_experiments/predictability/figures/'
    #plt.savefig(workingPath+file_path+'table_aggressiveBehavior_localidades'+str(name_experiment)+'_predict_measures_time_'+str(lenWindow)+'_crime_'+str(crime_level),dpi=300,bbox_inches = "tight")
    plt.show()

In [ ]:
def map_localidad(ax,df,col_localidad,col_vals,vmin=None,vmax=None):
    loc_geo=workingPath+"assets/localidades_polygon.json"
    loc_=gpd.read_file(loc_geo)
    loc_=loc_.merge(df,left_on='LocNombre',right_on=col_localidad)
    loc_.plot(cmap='viridis',edgecolor='white',column=col_vals,ax=ax,vmin=vmin,vmax=vmax,legend=True)

In [ ]:
def map_predictability(df_crime, crime_level, lenWindow,name_experiment):
    subdata = df_crime[df_crime['crime_level']==crime_level]
    subdata = subdata[subdata['lenWindow']==str(lenWindow)]
    subdata["predictability"] = pd.to_numeric(subdata["predictability"])
    
    fig, ax = plt.subplots(figsize=(12,12))
    map_localidad(ax,subdata,'localidad','predictability',vmin=0,vmax=0.45)
    ax.axis('off')
    file_path = 'periodicity_experiments/predictability/figures/'
    #plt.savefig(workingPath+file_path+'map_aggressiveBehavior_localidades'+str(name_experiment)+'_predictability_time_'+str(lenWindow)+'_levels_'+str(crime_level),dpi=300,bbox_inches = "tight")
    plt.show()

## Load data

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/06. verify_enrich_rnmc_12022020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [ ]:
df_input.TIPO_PRIORIZACION.unique()

In [ ]:
df_period = df_input[df_input['ANIO']!=2019]
df_period.columns

In [ ]:
df_period['date']=pd.to_datetime(df_period['FECHA'])
df_by_date = pd.DataFrame(df_period.groupby(['date','NOMBRE_LOCALIDAD']).size(),columns=["total_eventos"])

In [ ]:
df_by_date = df_by_date.reset_index().set_index('date')

In [ ]:
df_by_date

## Experiment to validate H1

In [ ]:
Levels=[3,5,10]
levelCategories = list(map(lambda x: str(x), Levels))
localidadesList = list(df_by_date.NOMBRE_LOCALIDAD.unique())
localidadesList.remove('SUMAPAZ')
timeWindows = [7]

In [ ]:
localidadesList

In [ ]:
for lenWindow in timeWindows:
    expName = '_localidades_rnmc_'
    periodName = '2017-2018'
    min_date_on_period = df_by_date.reset_index().date.min()
    max_date_on_period = df_by_date.reset_index().date.max()
    predValues = predictability_experiment_localidades(df_by_date,min_date_on_period,max_date_on_period,lenWindow,localidadesList,Levels,expName,periodName)
    df_prediction = pd.DataFrame(predValues, columns=['experiment_name','period','localidad','lenWindow','crime_level','predictability','contingency','constancy'])
    table_predictability_report(df_prediction,lenWindow,localidadesList,levelCategories,expName)
    #table_constancy_report(df_prediction,lenWindow,localidadesList,levelCategories,name_experiment)
    #table_contingency_report(df_prediction,lenWindow,localidadesList,levelCategories,name_experiment)
    crime_level = '3'
    map_predictability(df_prediction, crime_level, lenWindow, expName)

In [ ]:
ciudad_bolivar=df_by_date[df_by_date['NOMBRE_LOCALIDAD']=='CIUDAD BOLIVAR']
min_date_on_period = df_by_date.reset_index().date.min()
max_date_on_period = df_by_date.reset_index().date.max()
ciudad_bolivar = preprocess_df(ciudad_bolivar,min_date_on_period,max_date_on_period)
ciudad_bolivar.head(90).plot()

In [ ]:
ciudad_bolivar=df_by_date[df_by_date['NOMBRE_LOCALIDAD']=='CIUDAD BOLIVAR']
min_date_on_period = df_by_date.reset_index().date.min()
max_date_on_period = df_by_date.reset_index().date.max()
#ciudad_bolivar = preprocess_df(ciudad_bolivar,min_date_on_period,max_date_on_period)
ciudad_bolivar.head(90).plot()

In [ ]:
ciudad_bolivar.head(50)

In [ ]:
teusaquillo=df_by_date[df_by_date['NOMBRE_LOCALIDAD']=='TEUSAQUILLO']
min_date_on_period = df_by_date.reset_index().date.min()
max_date_on_period = df_by_date.reset_index().date.max()
teusaquillo = preprocess_df(teusaquillo,min_date_on_period,max_date_on_period)
teusaquillo.head(90).plot()